<a href="https://colab.research.google.com/github/nihald2000/Edgeai/blob/main/GenaiRAGAPP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# !pip install -q langchain langchain-google-genai  bs4 faiss-cpu
!pip install -q \
    langchain \
    faiss-cpu \
    langchain-community \
    langchain-google-genai \
    bs4 \
    google-ai-generativelanguage==0.6.15

In [13]:
import os
# from dotenv import load_dotenv
# load_dotenv()
from google.colab import userdata
userdata.get('GOOGLE_API_KEY')

# os.environ['GOOGLE_API_KEY']=os.getenv("GOOGLE_API_KEY")
## Langsmith Tracking
# os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
# os.environ["LANGCHAIN_TRACING_V2"]="true"
# os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")


# LangSmith Configuration
os.environ["LANGCHAIN_TRACING_V2"] = "true"
LANGCHAIN_API_KEY = userdata.get('LANGCHAIN_API_KEY')
if LANGCHAIN_API_KEY:
    os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
    print(" LangSmith API Key loaded.")
else:
    print("LANGCHAIN_API_KEY not found in Colab Secrets. LangSmith tracing will not work.")

# You can customize your LangSmith project name here
LANGCHAIN_PROJECT_NAME = "My GENAI RAG APP"
os.environ["LANGCHAIN_PROJECT"] = LANGCHAIN_PROJECT_NAME
print(f" LangSmith Project set to: {LANGCHAIN_PROJECT_NAME}")

# Google Gemini Configuration
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
if GOOGLE_API_KEY:
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("Google (Gemini) API Key loaded.")
else:
    print(" GOOGLE_API_KEY not found in Colab Secrets. Gemini calls will fail if attempted.")


 LangSmith API Key loaded.
 LangSmith Project set to: My GENAI RAG APP
Google (Gemini) API Key loaded.


In [22]:
#Data Inserstion
from langchain.document_loaders import WebBaseLoader

loader=WebBaseLoader("https://medium.com/@nihald2000/a-comprehensive-guide-to-formatting-a-64gb-usb-drive-to-fat32-af5fa92c9c4b")
loader


In [23]:
docs = loader.load()
docs

[Document(metadata={'source': 'https://medium.com/@nihald2000/a-comprehensive-guide-to-formatting-a-64gb-usb-drive-to-fat32-af5fa92c9c4b', 'title': 'A Comprehensive Guide to Formatting a 64GB USB Drive to FAT32 | by Nihal Degaon | Medium', 'description': 'Formatting a USB drive to FAT32 is a common requirement for compatibility with various devices, such as cameras, gaming consoles, or legacy systems. However, Windows’ built-in tools limit FAT32…', 'language': 'en'}, page_content='A Comprehensive Guide to Formatting a 64GB USB Drive to FAT32 | by Nihal Degaon | MediumSitemapOpen in appSign upSign inMedium LogoWriteSign upSign inA Comprehensive Guide to Formatting a 64GB USB Drive to FAT32Nihal DegaonFollow3 min read·Mar 19, 2025--ListenShareFormatted to FAT32Formatting a USB drive to FAT32 is a common requirement for compatibility with various devices, such as cameras, gaming consoles, or legacy systems. However, Windows’ built-in tools limit FAT32 formatting to drives of 32GB or less,

In [25]:
#Dividing data into smaller chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800,chunk_overlap=150)
documents = text_splitter.split_documents(docs)
documents

[Document(metadata={'source': 'https://medium.com/@nihald2000/a-comprehensive-guide-to-formatting-a-64gb-usb-drive-to-fat32-af5fa92c9c4b', 'title': 'A Comprehensive Guide to Formatting a 64GB USB Drive to FAT32 | by Nihal Degaon | Medium', 'description': 'Formatting a USB drive to FAT32 is a common requirement for compatibility with various devices, such as cameras, gaming consoles, or legacy systems. However, Windows’ built-in tools limit FAT32…', 'language': 'en'}, page_content='A Comprehensive Guide to Formatting a 64GB USB Drive to FAT32 | by Nihal Degaon | MediumSitemapOpen in appSign upSign inMedium LogoWriteSign upSign inA Comprehensive Guide to Formatting a 64GB USB Drive to FAT32Nihal DegaonFollow3 min read·Mar 19, 2025--ListenShareFormatted to FAT32Formatting a USB drive to FAT32 is a common requirement for compatibility with various devices, such as cameras, gaming consoles, or legacy systems. However, Windows’ built-in tools limit FAT32 formatting to drives of 32GB or less,

In [28]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


In [32]:
from langchain_community.vectorstores import FAISS
vectorstoresdb = FAISS.from_documents(documents,embeddings)
vectorstoresdb

In [33]:
## Query From a vector db
query="Third-party utilities bypass Windows’ 32GB restriction"
result=vectorstoresdb.similarity_search(query)
result[0].page_content

'Windows’ 32GB restriction and offer a quick solution:Rufus:Download Rufus from rufus.ie.Insert your USB drive.Open Rufus and select your USB under the “Device” field.Under “File System,” choose FAT32.Click Start and confirm the formatting process.FAT32 Format Tool:Download the utility from ridgecrop.co.uk.Run the executable, select your USB drive, and click Start.These tools are particularly useful for drives larger than 32GB because they circumvent Windows’ limitations.Method 2: Using PowerShellIf you prefer not to install extra software, you can use PowerShell. Note that formatting a large drive this way may be considerably slower:Open PowerShell as Administrator.Execute the following command (replacing X: with your USB drive letter):format /FS:FAT32 X:Be prepared — this method can take'

In [34]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm=ChatGoogleGenerativeAI(model="gemini-2.5-flash-preview-04-17")

In [35]:
#Retrival chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| ChatGoogleGenerativeAI(model='models/gemini-2.5-flash-preview-04-17', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7e2c08552490>, default_metadata=())
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [36]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"LangSmith has two usage limits: total traces and extended",
    "context":[Document(page_content="LangSmith has two usage limits: total traces and extended traces. These correspond to the two metrics we've been tracking on our usage graph. ")]
})

'Based on the provided context, LangSmith has two usage limits:\n1.  Total traces\n2.  Extended traces\n\nThese correspond to the two metrics tracked on their usage graph.'

In [37]:
vectorstoresdb

In [39]:
retriever=vectorstoresdb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)
retrieval_chain


RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7e2c0838f150>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
      

In [40]:
## Get the response form the LLM
response=retrieval_chain.invoke({"input":"Linux offers robust command-line tools"})
response['answer']

'Based on the provided context, the article was published on **Mar 19, 2025**.'

In [41]:
response

{'input': 'Linux offers robust command-line tools',
 'context': [Document(id='d496a183-3348-47a4-b916-14e20cae15f6', metadata={'source': 'https://medium.com/@nihald2000/a-comprehensive-guide-to-formatting-a-64gb-usb-drive-to-fat32-af5fa92c9c4b', 'title': 'A Comprehensive Guide to Formatting a 64GB USB Drive to FAT32 | by Nihal Degaon | Medium', 'description': 'Formatting a USB drive to FAT32 is a common requirement for compatibility with various devices, such as cameras, gaming consoles, or legacy systems. However, Windows’ built-in tools limit FAT32…', 'language': 'en'}, page_content='Since your output indicates that /dev/sdb1 is not mounted, you can skip this step. If it were mounted, you would use:sudo umount /dev/sdb1Run the Formatting Command:sudo mkfs.fat -F 32 /dev/sdb1Verify the Format:lsblk -fMount the USB (Optional)sudo mkdir -p /mnt/usbsudo mount /dev/sdb1 /mnt/usbAlternatively, you can use a graphical tool like GParted if you prefer a GUI:sudo apt install gparted -yRun it w

In [42]:
response['context']

[Document(id='d496a183-3348-47a4-b916-14e20cae15f6', metadata={'source': 'https://medium.com/@nihald2000/a-comprehensive-guide-to-formatting-a-64gb-usb-drive-to-fat32-af5fa92c9c4b', 'title': 'A Comprehensive Guide to Formatting a 64GB USB Drive to FAT32 | by Nihal Degaon | Medium', 'description': 'Formatting a USB drive to FAT32 is a common requirement for compatibility with various devices, such as cameras, gaming consoles, or legacy systems. However, Windows’ built-in tools limit FAT32…', 'language': 'en'}, page_content='Since your output indicates that /dev/sdb1 is not mounted, you can skip this step. If it were mounted, you would use:sudo umount /dev/sdb1Run the Formatting Command:sudo mkfs.fat -F 32 /dev/sdb1Verify the Format:lsblk -fMount the USB (Optional)sudo mkdir -p /mnt/usbsudo mount /dev/sdb1 /mnt/usbAlternatively, you can use a graphical tool like GParted if you prefer a GUI:sudo apt install gparted -yRun it with:sudo gpartedDealing with Corporate USB RestrictionsIn some c